In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.pyplot import imshow
from tqdm import tqdm
import torch
from torchsummary import summary

from collections import namedtuple, defaultdict
from pathlib import Path
import time

%matplotlib inline
rcParams['figure.figsize'] = (10, 15)

In [7]:
import sys
sys.path.append('../../')
import os

In [8]:
from src.constructor.config_structure import TrainConfigParams
from src.registry import TASKS
from src.constructor.data import create_dataset
from train import load_config

In [12]:
config_path = '../configs/detection.yml'
config_yaml = load_config(config_path)
config = TrainConfigParams(**config_yaml)

In [13]:
data_params = config.data
dataset_name = data_params.train_params.name
common_params = data_params.common_params
other_params = data_params.train_params
# other_params = data_params.valid_params
# other_params = data_params.test_params

dataset = create_dataset(dataset_name, common_params, other_params)

In [14]:
dataset[0]

{'input': tensor([[[-0.1785, -0.1544,  0.0711,  ..., -1.0000, -1.0000, -1.0000],
          [-0.0358, -0.1147, -0.1451,  ..., -0.9985, -0.9985, -1.0000],
          [-0.1554, -0.1583, -0.3662,  ..., -0.9922, -0.9922, -0.9976],
          ...,
          [-0.4858, -0.4453, -0.5654,  ..., -0.9893, -0.9814, -0.9863],
          [-0.4561, -0.4583, -0.5830,  ..., -0.9531, -0.9766, -0.9644],
          [-0.4326, -0.4922, -0.5806,  ..., -0.9009, -0.9507, -0.9937]],
 
         [[ 0.8555,  0.7935,  0.6802,  ...,  0.1807,  0.1838,  0.1838],
          [ 0.7148,  0.6162,  0.5059,  ...,  0.1814,  0.1750,  0.1736],
          [ 0.5415,  0.4895,  0.4578,  ...,  0.1686,  0.1686,  0.1632],
          ...,
          [-0.3557, -0.3713, -0.5005,  ...,  0.1142, -0.0066, -0.1313],
          [-0.3633, -0.4109, -0.5361,  ...,  0.1848,  0.0368, -0.0970],
          [-0.3647, -0.4636, -0.5483,  ...,  0.3584,  0.1464,  0.0500]],
 
         [[ 0.6870,  0.6870,  0.7744,  ...,  1.0000,  1.0000,  1.0000],
          [ 0.7031,

In [15]:
from torch.utils.data import DataLoader

collate_fn = dataset.collate_fn

loader = DataLoader(dataset=dataset, batch_size=50,
                        num_workers=6,
                        collate_fn=collate_fn)

In [16]:
next(iter(loader))

{'input': tensor([[[[-1.7847e-01, -1.5442e-01,  7.1106e-02,  ..., -1.0000e+00,
            -1.0000e+00, -1.0000e+00],
           [-3.5797e-02, -1.1469e-01, -1.4514e-01,  ..., -9.9854e-01,
            -9.9854e-01, -1.0000e+00],
           [-1.5540e-01, -1.5833e-01, -3.6621e-01,  ..., -9.9219e-01,
            -9.9219e-01, -9.9756e-01],
           ...,
           [-4.8584e-01, -4.4531e-01, -5.6543e-01,  ..., -9.8926e-01,
            -9.8145e-01, -9.8633e-01],
           [-4.5605e-01, -4.5825e-01, -5.8301e-01,  ..., -9.5312e-01,
            -9.7656e-01, -9.6436e-01],
           [-4.3262e-01, -4.9219e-01, -5.8057e-01,  ..., -9.0088e-01,
            -9.5068e-01, -9.9365e-01]],
 
          [[ 8.5547e-01,  7.9346e-01,  6.8018e-01,  ...,  1.8066e-01,
             1.8384e-01,  1.8384e-01],
           [ 7.1484e-01,  6.1621e-01,  5.0586e-01,  ...,  1.8140e-01,
             1.7505e-01,  1.7358e-01],
           [ 5.4150e-01,  4.8950e-01,  4.5776e-01,  ...,  1.6858e-01,
             1.6858e-01,  1.63